# Ансамблевые методы

In [1]:
import cv2
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from imutils import paths
import numpy as np

### Загрузка выборки изображений кошек и собак

In [2]:
imagePaths = sorted(list(paths.list_images('train_task4\\train')))

In [28]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def classification(image_path: list[str]) -> list[int]:
    y = []
    for image in imagePaths:
        if ('cat' in image):
            y.append(1)
        elif ('dog' in image):
            y.append(0)
        else:
            y.append(-1)
    return y

In [29]:
X = [extract_histogram(cv2.imread(image)) for image in imagePaths]
y = np.array(classification(imagePaths))

In [30]:
SAMPLE_LIST = ["test_task\\test\\cat.1009.jpg",
               "test_task\\test\\cat.1015.jpg",
               "test_task\\test\\cat.1028.jpg",
               "test_task\\test\\dog.1014.jpg"]

## Обучение базовых алгоритмов на исходном наборе данных

Классификатор с мягким зазором

In [31]:
svm = LinearSVC(C = 1.74, random_state = 220)
svm.fit(X,y)

LinearSVC(C=1.74, random_state=220)

Бэггинг деревьев принятия решений

In [32]:
bagging = BaggingClassifier(DecisionTreeClassifier(criterion = 'entropy', 
                                                   min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 220),n_estimators = 19,random_state = 220)
bagging.fit(X,y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_leaf_nodes=20,
                                                        min_samples_leaf=10,
                                                        random_state=220),
                  n_estimators=19, random_state=220)

Случайный лес

In [33]:
forest = RandomForestClassifier(n_estimators = 19, criterion = 'entropy',min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 220)
forest.fit(X,y)

RandomForestClassifier(criterion='entropy', max_leaf_nodes=20,
                       min_samples_leaf=10, n_estimators=19, random_state=220)

## Инициализация метаалгоритма

Логистическая регрессия

In [34]:
log_reg = LogisticRegression(solver='lbfgs',random_state = 220)

## Обучение модели стэкинга

Используется 2-fold (`cv = 2`) кросс-валидация

In [35]:
base_estimators = [('SVM', svm), ('Bagging DT', bagging), ('DecisionForest', forest)]
sclf = StackingClassifier(estimators=base_estimators, final_estimator=log_reg, cv=2)
sclf.fit(X,y)

StackingClassifier(cv=2,
                   estimators=[('SVM', LinearSVC(C=1.74, random_state=220)),
                               ('Bagging DT',
                                BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                        max_leaf_nodes=20,
                                                                                        min_samples_leaf=10,
                                                                                        random_state=220),
                                                  n_estimators=19,
                                                  random_state=220)),
                               ('DecisionForest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_leaf_nodes=20,
                                                       min_samples_leaf=10,
       

### Доля правильной классификации на тренировочных данных

In [36]:
sclf.score(X,y)

0.851

### Вероятности отнесения изображений к классам с помощью обученного метаалгоритма

In [37]:
Test_sample = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[0]))).reshape(1,-1)
print(sclf.predict_proba(Test_sample))
Test_sample1 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[1]))).reshape(1,-1)
print(sclf.predict_proba(Test_sample1))
Test_sample2 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[2]))).reshape(1,-1)
print(sclf.predict_proba(Test_sample2))
Test_sample3 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[3]))).reshape(1,-1)
print(sclf.predict_proba(Test_sample3))

[[0.52571911 0.47428089]]
[[0.3207964 0.6792036]]
[[0.51493032 0.48506968]]
[[0.32412333 0.67587667]]


In [38]:
Test_sample = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[0]))).reshape(1,-1)
print(sclf.predict(Test_sample))
Test_sample1 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[1]))).reshape(1,-1)
print(sclf.predict(Test_sample1))
Test_sample2 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[2]))).reshape(1,-1)
print(sclf.predict(Test_sample2))
Test_sample3 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[3]))).reshape(1,-1)
print(sclf.predict(Test_sample3))

[0]
[1]
[0]
[1]
